<a href="https://colab.research.google.com/github/hufsaim/T10306201/blob/master/notebook_2024/Lab08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install pyradiomics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Discarding https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz (from https://pypi.org/simple/pyradiomics/): Requested pyradiomics from https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz has inconsistent version: expected '3.1.0', but metadata has '3.0.1a1'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.

In [2]:
import radiomics
from radiomics import featureextractor
import matplotlib.pyplot as plt
import SimpleITK as sitk
import nibabel as nib
import pandas as pd

In [3]:
# First define the settings
settings = {}
settings['binWidth'] = 20
settings['sigma'] = [1]
settings['normalize']=True
settings['geometryTolerance']=1
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor(**settings)  # ** 'unpacks' the dictionary in the function call
# Enable a filter (in addition to the 'Original' filter already enabled)
#extractor.enableImageTypeByName('LoG')
#extractor.enableImageTypeByName('Wavelet')
print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)  # Still the default parameters
print('Enabled features:\n\t', extractor.enabledFeatures)  # Still the default parameters


INFO:radiomics.featureextractor:No valid config parameter, using defaults: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}
INFO:radiomics.featureextractor:Enabled image types: {'Original': {}}
INFO:radiomics.featureextractor:Enabled features: {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}
INFO:radiomics.featureextractor:Applying custom setting overrides: {'binWidth': 20, 'sigma': [1], 'normalize': True, 'geometryTolerance': 1}


Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': True, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 20, 'sigma': [1], 'geometryTolerance': 1}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}


In [4]:
h1 = nib.load('H_t1ce.nii.gz')
img = h1.get_fdata()
img.shape

(240, 240, 155)

In [5]:
h2 = nib.load('H_seg.nii.gz')
seg = h2.get_fdata()
seg.shape

(240, 240, 155)

In [6]:
image = sitk.GetImageFromArray(img)
mask = sitk.GetImageFromArray(seg)

In [7]:
features = extractor.execute(image,mask)

INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing ngtdm


In [8]:
features

OrderedDict([('diagnostics_Versions_PyRadiomics', 'v3.0.1'),
             ('diagnostics_Versions_Numpy', '1.26.4'),
             ('diagnostics_Versions_SimpleITK', '2.4.0'),
             ('diagnostics_Versions_PyWavelet', '1.7.0'),
             ('diagnostics_Versions_Python', '3.10.12'),
             ('diagnostics_Configuration_Settings',
              {'minimumROIDimensions': 2,
               'minimumROISize': None,
               'normalize': True,
               'normalizeScale': 1,
               'removeOutliers': None,
               'resampledPixelSpacing': None,
               'interpolator': 'sitkBSpline',
               'preCrop': False,
               'padDistance': 5,
               'distances': [1],
               'force2D': False,
               'force2Ddimension': 0,
               'resegmentRange': None,
               'label': 1,
               'additionalInfo': True,
               'binWidth': 20,
               'sigma': [1],
               'geometryTolerance': 1}),
 

In [9]:
feature_names = list((filter(lambda k: k.startswith("original_"),features)))
feature_names

['original_shape_Elongation',
 'original_shape_Flatness',
 'original_shape_LeastAxisLength',
 'original_shape_MajorAxisLength',
 'original_shape_Maximum2DDiameterColumn',
 'original_shape_Maximum2DDiameterRow',
 'original_shape_Maximum2DDiameterSlice',
 'original_shape_Maximum3DDiameter',
 'original_shape_MeshVolume',
 'original_shape_MinorAxisLength',
 'original_shape_Sphericity',
 'original_shape_SurfaceArea',
 'original_shape_SurfaceVolumeRatio',
 'original_shape_VoxelVolume',
 'original_firstorder_10Percentile',
 'original_firstorder_90Percentile',
 'original_firstorder_Energy',
 'original_firstorder_Entropy',
 'original_firstorder_InterquartileRange',
 'original_firstorder_Kurtosis',
 'original_firstorder_Maximum',
 'original_firstorder_MeanAbsoluteDeviation',
 'original_firstorder_Mean',
 'original_firstorder_Median',
 'original_firstorder_Minimum',
 'original_firstorder_Range',
 'original_firstorder_RobustMeanAbsoluteDeviation',
 'original_firstorder_RootMeanSquared',
 'original

In [14]:
df1 = pd.DataFrame()
for j in range(len(feature_names)):
  print(j,feature_names[j],features[feature_names[j]])
  new_row = {"feature_name":feature_names[j], "H":features[feature_names[j]]}
  df1 = pd.concat([df1, pd.DataFrame([new_row])], ignore_index=True)

0 original_shape_Elongation 0.7705221899156938
1 original_shape_Flatness 0.4456204321340164
2 original_shape_LeastAxisLength 8.942992981667214
3 original_shape_MajorAxisLength 20.068633161276786
4 original_shape_Maximum2DDiameterColumn 25.553864678361276
5 original_shape_Maximum2DDiameterRow 20.0
6 original_shape_Maximum2DDiameterSlice 21.840329667841555
7 original_shape_Maximum3DDiameter 27.910571473905726
8 original_shape_MeshVolume 339.4583333333333
9 original_shape_MinorAxisLength 15.463327172041701
10 original_shape_Sphericity 0.3280588293045878
11 original_shape_SurfaceArea 717.3372388711193
12 original_shape_SurfaceVolumeRatio 2.113181997410932
13 original_shape_VoxelVolume 413.0
14 original_firstorder_10Percentile 1.7756253364023524
15 original_firstorder_90Percentile 2.8247011799850297
16 original_firstorder_Energy 2380.4530635283995
17 original_firstorder_Entropy -3.203426503814917e-16
18 original_firstorder_InterquartileRange 0.5057083553680597
19 original_firstorder_Kurtosi

In [15]:
df1

,feature_name,H
0,original_shape_Elongation,0.770522
1,original_shape_Flatness,0.44562
2,original_shape_LeastAxisLength,8.942993
3,original_shape_MajorAxisLength,20.068633
4,original_shape_Maximum2DDiameterColumn,25.553864678361276
...,...,...
102,original_ngtdm_Busyness,0.0
103,original_ngtdm_Coarseness,1000000.0
104,original_ngtdm_Complexity,0.0
105,original_ngtdm_Contrast,0.0


In [16]:
df1.to_csv("H_features.csv")